# 250401_01_docking_data_processing 

# 1.00 Preview input

In [1]:
import pandas as pd

# Define the file path
file_path = '00_cdkl5_targets.xlsx'

# Load the first sheet of the Excel file
df = pd.read_excel(file_path, sheet_name=0)

# Display the entire DataFrame
print(df.to_string())  # Converts the entire DataFrame to a string for full display

    SL UniProt                   Gene  pSite Consensus_motif_(R-P-X-[S/T]-[A/G/P/S])                                                             Protein                                                                                                                                                                                                                                Protein function in relation to CDKL5                                    Model system (e.g. animal, cell line, etc.)   Full length or kinase domain           In vitro or in vivo                                                                                                                                                                                                                                                                                              Method of identification        Citation
0    1  O76039                  CDKL5   Y171                                     TEY                                    

# 01 CDKL5 Target Motif Extraction Script

This Python script processes an Excel file containing CDKL5 target information, retrieves protein sequences from UniProt, and extracts consensus motifs or sequences around phosphorylation sites (pSites).

## Purpose

The script aims to:

1.  Retrieve protein sequences from UniProt based on UniProt IDs.
2.  Identify consensus motifs matching the pattern `RP.[ST][AGPS]` within the sequences.
3.  If no consensus motif is found, extract a 5-amino acid sequence around the provided pSite.
4.  Record the extracted motifs, their ranges, and the extraction method used.
5.  Output the results to a new Excel file.

## Input Excel File Format

The input Excel file should contain the following columns:

* **UniProt:** UniProt accession ID (e.g., P56524).
* **pSite:** Phosphorylation site (e.g., S632).
* **Other columns:** Any additional information related to the targets.

## Output Excel File Format

The output Excel file will contain all the original columns from the input file, plus the following:

* **Consensus Motifs Found:** The extracted consensus motif(s) or the sequence around the pSite.
* **Consensus Motif Range:** The range of amino acid positions for the extracted motif.
* **Extraction Method:** Indicates how the motif was extracted (Consensus Motif, pSite, None, pSite Extraction Failed, Sequence Retrieval Failed).

## Script Details

* **`get_uniprot_sequence(uniprot_id)`:** Retrieves protein sequences from UniProt.
* **`find_consensus_motifs(sequence)`:** Finds consensus motifs using the regular expression `RP.[ST][AGPS]`.
* **`get_motif_range(motif)`:** Extracts the range of amino acid positions for a motif.
* **`extract_motif_around_psite(sequence, psite_aa, psite_pos_excel, motif_length=5)`:** Extracts a sequence around the pSite.
    * `psite_aa`: the amino acid, e.g., 'S'
    * `psite_pos_excel`: the position from the excel file, e.g. '632'
    * `motif_length`: the length of the extracted sequence (default: 5).

## Error Handling

* The script handles cases where UniProt sequences cannot be retrieved.
* It provides informative error messages for pSite extraction failures.
* It identifies whether the extracted motif was found using the consensus motif or the pSite.

## Example Output

| UniProt | pSite | Consensus Motifs Found | Consensus Motif Range | Extraction Method |
| ------- | ----- | ---------------------- | --------------------- | ----------------- |
| P56524  | S632  | ['RPLSRAQ']           | 631-637               | pSite             |
| Q92974  | S122  | ['RPSSA']              | 119-123               | Consensus Motif   |
| P84022  |       | []                     |                       | None              |
| O76039  | Y171  | []                     |                       | pSite Extraction Failed: pSite amino acid not found in sequence |

In [2]:
import requests
import re
import pandas as pd
from tqdm import tqdm

def get_uniprot_sequence(uniprot_id):
    """Retrieves the protein sequence from UniProt."""
    url = f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta"
    try:
        response = requests.get(url)
        response.raise_for_status()
        fasta_data = response.text
        sequence = "".join(fasta_data.splitlines()[1:])
        return sequence
    except requests.exceptions.RequestException as e:
        return None

def find_consensus_motifs(sequence):
    """Finds consensus motifs in the protein sequence."""
    motif_pattern = re.compile(r"RP.[ST][AGPS]")
    return [(match.start() + 1, match.group(0)) for match in motif_pattern.finditer(sequence)]

def get_motif_range(motif):
    """Extracts the range from a single motif."""
    return f"{motif[0]}-{motif[0] + len(motif[1]) - 1}"

def extract_motif_around_psite(sequence, psite_aa, psite_pos_excel, motif_length=5):
    """Extracts motif around the pSite with specified length."""
    if not isinstance(psite_aa, str):
        return None, None, "Invalid pSite amino acid"

    psite_aa_upper = psite_aa.upper()
    psite_positions_in_seq = [i + 1 for i, aa in enumerate(sequence.upper()) if aa == psite_aa_upper]

    if not psite_positions_in_seq:
        return None, None, "pSite amino acid not found in sequence"

    # Find the closest matching pSite in the sequence to the provided position.
    closest_psite_pos = min(psite_positions_in_seq, key=lambda x: abs(x - int(psite_pos_excel)))

    start = max(0, closest_psite_pos - (motif_length // 2))
    end = min(len(sequence), closest_psite_pos + (motif_length - (motif_length // 2)))
    extracted_motif = sequence[start - 1:end - 1] #index correction
    extracted_range = f"{start}-{end}"
    return extracted_motif, extracted_range, None

# Load data from Excel
excel_path = "00_cdkl5_targets.xlsx"
df = pd.read_excel(excel_path)
df.columns = df.columns.str.strip()

results = []

# Process each entry
for i, row in tqdm(df.iterrows(), total=len(df)):
    uniprot_id = row['UniProt']
    psite = row['pSite']
    sequence = get_uniprot_sequence(uniprot_id)

    if sequence:
        motifs = find_consensus_motifs(sequence)
        if motifs:
            for motif in motifs:
                result_row = row.to_dict()
                result_row['Consensus Motifs Found'] = str([motif])
                result_row['Consensus Motif Range'] = get_motif_range(motif)
                result_row['Extraction Method'] = 'Consensus Motif'
                results.append(result_row)
        elif pd.notna(psite) and psite != "":
            psite_aa = psite[0]  # Get the amino acid (e.g., 'S')
            psite_pos_excel = psite[1:] # get position from excel.
            extracted_motif, extracted_range, error = extract_motif_around_psite(sequence, psite_aa, psite_pos_excel)
            if extracted_motif:
                result_row = row.to_dict()
                result_row['Consensus Motifs Found'] = str([extracted_motif])
                result_row['Consensus Motif Range'] = extracted_range
                result_row['Extraction Method'] = 'pSite'
                results.append(result_row)
            else:
                result_row = row.to_dict()
                result_row['Consensus Motifs Found'] = '[]'
                result_row['Consensus Motif Range'] = ''
                result_row['Extraction Method'] = f'pSite Extraction Failed: {error}'
                results.append(result_row)
        else:
            result_row = row.to_dict()
            result_row['Consensus Motifs Found'] = '[]'
            result_row['Consensus Motif Range'] = ''
            result_row['Extraction Method'] = 'None'
            results.append(result_row)
    else:
        result_row = row.to_dict()
        result_row['Consensus Motifs Found'] = ''
        result_row['Consensus Motif Range'] = ''
        result_row['Extraction Method'] = 'Sequence Retrieval Failed'
        results.append(result_row)

df_results = pd.DataFrame(results)
df_results.to_excel("01_updated_consensus_motifs_psite_position.xlsx", index=False)
print("✅ Analysis complete. Updated file saved as 'updated_consensus_motifs_psite_position.xlsx'")

100%|██████████| 27/27 [00:25<00:00,  1.06it/s]

✅ Analysis complete. Updated file saved as 'updated_consensus_motifs_psite_position.xlsx'


# 02 Get PDB using uniprot
get available protein structure into excel both from protein data bank and alphafold corresponding to each uniprot ID

In [3]:
import pandas as pd
import requests

# Define file paths
input_file = "01_updated_consensus_motifs_psite_position.xlsx"  # Input file
output_file = "02_updated_consensus_motifs_with_pdb.xlsx"  # Updated output file

# Load the Excel sheet
df = pd.read_excel(input_file, sheet_name=0)

# Check if required columns exist
required_columns = ["UniProt", "Gene", "Protein", "pSite", "Consensus_motif_(R-P-X-[S/T]-[A/G/P/S])"]
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"Missing required column: {col}")

# Function to fetch PDB structural data from UniProt
def get_uniprot_structures(uniprot_id):
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"
    response = requests.get(url)
    
    structures = []
    if response.status_code == 200:
        data = response.json()
        if "uniProtKBCrossReferences" in data:
            for entry in data["uniProtKBCrossReferences"]:
                if entry["database"] == "PDB":
                    pdb_id = entry["id"]
                    method = next((prop["value"] for prop in entry.get("properties", []) if prop["key"] == "Method"), "N/A")
                    resolution = next((prop["value"] for prop in entry.get("properties", []) if prop["key"] == "Resolution"), "N/A")
                    chains_positions = next((prop["value"] for prop in entry.get("properties", []) if prop["key"] == "Chains"), "N/A")

                    chain_list, position_list = [], []
                    if chains_positions != "N/A":
                        for part in chains_positions.split(", "):
                            if "=" in part:
                                chain, position = part.split("=")
                                chain_list.append(chain.strip())
                                position_list.append(position.strip())
                            else:
                                chain_list.append(part.strip())
                                position_list.append("N/A")

                    structures.append({
                        "PDB_ID": pdb_id,
                        "Method": method,
                        "Resolution": resolution,
                        "Chain": ", ".join(chain_list),
                        "Positions": ", ".join(position_list),
                        "Source": "UniProt-PDB"
                    })
    
    return structures

# Function to add AlphaFold structure
def get_alphafold_structure(uniprot_id):
    alphafold_id = f"AF-{uniprot_id}-F1"
    
    return [{
        "PDB_ID": alphafold_id,
        "Method": "AlphaFold Prediction",
        "Resolution": "N/A",
        "Chain": "N/A",
        "Positions": "N/A",
        "Source": "AlphaFold DB"
    }]

# Collect all structures and map them to input data
pdb_data = []
for _, row in df.iterrows():
    uniprot_id = row["UniProt"]

    # Fetch UniProt PDB structures
    pdb_results = get_uniprot_structures(uniprot_id)

    # Always add AlphaFold prediction
    pdb_results.extend(get_alphafold_structure(uniprot_id))

    for pdb_entry in pdb_results:
        pdb_entry.update({
            "UniProt": uniprot_id
        })
        pdb_data.append(pdb_entry)

# Convert to DataFrame
pdb_df = pd.DataFrame(pdb_data)

# Merge original data with PDB structures
df_updated = df.merge(pdb_df, on="UniProt", how="left")

# Save the updated DataFrame to Excel
df_updated.to_excel(output_file, index=False)

print(f"Updated file saved as {output_file}")


Updated file saved as 02_updated_consensus_motifs_with_pdb.xlsx


# 03 Alphafold entries only
- Upon investigation most PDB structures missing pSite
- Thus focus on alphafold entries only
- get updated excel sheet with alphafold entreis only
- Downlaod alphafold structures

In [4]:
import pandas as pd
import requests
import os

# Define file paths
input_file = "01_updated_consensus_motifs_psite_position.xlsx"  # Input Excel file
output_file = "03_updated_alphafold_entries.xlsx"  # Output Excel file
structure_folder = "03_alphafold_structures/"  # Folder to save AlphaFold PDB files

# Ensure the structure folder exists
os.makedirs(structure_folder, exist_ok=True)

# Load the Excel sheet
df = pd.read_excel(input_file, sheet_name=0)

# Function to generate AlphaFold PDB ID and URL
def get_alphafold_pdb_info(uniprot_id):
    pdb_id = f"AF-{uniprot_id}-F1"
    pdb_url = f"https://alphafold.ebi.ac.uk/files/{pdb_id}-model_v4.pdb"
    return pdb_id, pdb_url

# Function to download AlphaFold PDB structure
def download_alphafold_pdb(uniprot_id):
    pdb_id, pdb_url = get_alphafold_pdb_info(uniprot_id)
    output_path = os.path.join(structure_folder, f"{pdb_id}.pdb")
    
    response = requests.get(pdb_url)
    if response.status_code == 200:
        with open(output_path, "wb") as f:
            f.write(response.content)
        return output_path  # Return file path if successful
    else:
        return "Download failed"  # Indicate failure

# Process each UniProt ID
df["AlphaFold_PDB_ID"], df["AlphaFold_PDB_URL"] = zip(*df["UniProt"].apply(get_alphafold_pdb_info))
df["AlphaFold_PDB_File_Path"] = df["UniProt"].apply(download_alphafold_pdb)

# Save the updated DataFrame to an Excel file
df.to_excel(output_file, index=False)

print(f"Updated AlphaFold entries saved to {output_file}")


Updated AlphaFold entries saved to 03_updated_alphafold_entries.xlsx
